<a href="https://colab.research.google.com/github/unburied/DS-Unit-2-Linear-Models/blob/master/Doing_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Get/Split/Model(basic)


In [1]:
import pandas as pd
import numpy as np

In [2]:
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/nyc/nyc-rent-2016.csv'

df = pd.read_csv(WEB)

In [3]:
#View Columns
df.head().transpose()

,0,1,2,3,4
bathrooms,1.5,1,1,1,1
bedrooms,3,2,1,1,4
created,2016-06-24 07:54:24,2016-06-12 12:19:27,2016-04-17 03:26:41,2016-04-18 02:22:02,2016-04-28 01:32:41
description,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,,"Top Top West Village location, beautiful Pre-w...",Building Amenities - Garage - Garden - fitness...,Beautifully renovated 3 bedroom flex 4 bedroom...
display_address,Metropolitan Avenue,Columbus Avenue,W 13 Street,East 49th Street,West 143rd Street
latitude,40.7145,40.7947,40.7388,40.7539,40.8241
longitude,-73.9425,-73.9667,-74.0018,-73.9677,-73.9493
price,3000,5465,2850,3275,3350
street_address,792 Metropolitan Avenue,808 Columbus Avenue,241 W 13 Street,333 East 49th Street,500 West 143rd Street
interest_level,medium,low,high,low,low


In [4]:
#convert time string object to datetime object and set as index
df['created'] = pd.to_datetime(df.created, infer_datetime_format= True)

df = df.set_index('created')
df.head()

,bathrooms,bedrooms,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,...,high_speed_internet,balcony,swimming_pool,new_construction,exclusive,terrace,loft,garden_patio,common_outdoor_space,wheelchair_access
created,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,...,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,...,0,0,0,0,0,0,0,0,0,0
2016-04-17 03:26:41,1.0,1,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,...,0,0,0,0,0,0,0,0,0,0
2016-04-18 02:22:02,1.0,1,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,...,0,0,0,0,0,0,0,0,0,0
2016-04-28 01:32:41,1.0,4,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#Split Data Based on date
train = df[df.index.month < 6]
test = df[df.index.month == 6]

train.shape, test.shape

((31515, 33), (16785, 33))

In [6]:
#extract target values from df
y_train = train.pop('price')
y_test = test.pop('price')

y_train.shape, y_test.shape

((31515,), (16785,))

In [7]:
#get a baseline model
baseline = np.full_like(y_test, y_train.mean())

##Baseline

In [8]:
#get metric to test baseline 
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, baseline)

1052.5193327375632

##Upgrade Baseline Model


In [9]:
#import and instantiate model
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [10]:
#select features to model
#here we will start with all numerical features and exclude any objects
features = list(train.dtypes[train.dtypes != 'object'].index)

X_train = train[features]
X_test = test[features]

In [11]:
#Train model and store result
#Will be updating X_train and X_test so dont need to pass as new values 
def score(train,test):
  model.fit(train, y_train)
  y_pred = model.predict(test)

  return mean_absolute_error(y_test, y_pred)
  

In [12]:
score(X_train, X_test)

618.6796864572018

##Upgraded Baseline

In [13]:
better_baseline = score(X_train, X_test)
print(better_baseline)

618.6796864572018


##Feature Selection
  

###Numerical Features


In [14]:
#Iterate over features to determine which have the highest impact

impact = {}

for feature in features:
  model.fit(train[[feature]], y_train)
  y_pred = model.predict(test[[feature]])
  impact[feature] = mean_absolute_error(y_test, y_pred)
                
print(impact)

{'bathrooms': 826.8186291735684, 'bedrooms': 865.7153554398519, 'latitude': 1048.59143416186, 'longitude': 989.0487004130676, 'elevator': 1021.1914238867432, 'cats_allowed': 1049.6548133279234, 'hardwood_floors': 1050.4749022675514, 'dogs_allowed': 1049.2944534089013, 'doorman': 1002.9906206520152, 'dishwasher': 1026.0537865120186, 'no_fee': 1044.9120965283466, 'laundry_in_building': 1052.8562017778665, 'fitness_center': 1020.8674997057973, 'pre-war': 1052.1651020895429, 'laundry_in_unit': 1028.7246629448905, 'roof_deck': 1044.951424200772, 'outdoor_space': 1038.9376029999073, 'dining_room': 1029.926584738313, 'high_speed_internet': 1047.836861804129, 'balcony': 1040.7962982352733, 'swimming_pool': 1046.0479246061477, 'new_construction': 1052.4032354966687, 'exclusive': 1052.2622667593637, 'terrace': 1042.9198257172632, 'loft': 1051.8527496972874, 'garden_patio': 1046.3402013438135, 'common_outdoor_space': 1051.7843743981512, 'wheelchair_access': 1051.120211011886}


In [15]:
#structure as a series and sort. extracting top 5 values

impact_table = pd.Series(impact)
impact_table = impact_table.sort_values()

impact_table.head()

bathrooms          826.818629
bedrooms           865.715355
longitude          989.048700
doorman           1002.990621
fitness_center    1020.867500
dtype: float64

In [16]:
#Get number of effective features that matches or beats better_baseline

index = 0 
extract = {}
prev_mae = 0

for index in range(1,len(impact_table)):
  features = impact_table.index[0:index]
  
  if index == 0:
      model.fit(train[[features]], y_train)
      y_pred = model.predict(test[[features]])
      mae = mean_absolute_error(y_test, y_pred)
      print(mae)
  else:
      model.fit(train[features], y_train)
      y_pred = model.predict(test[features])
      mae = mean_absolute_error(y_test, y_pred)
      print(mae)
      
  #Find features whose effect on score is greater than 1
  effect = abs(mae - prev_mae)
  if effect > 1:
    extract[impact_table.index[index]] = effect          
    prev_mae = mae
    
print(extract)

826.8186291735684
754.5508722934986
667.2933147298323
635.1528773589963
635.1229219475481
634.375268871983
634.0690256373646
632.0777436135901
632.0763605880953
632.3628533828015
632.0742813294667
631.995876445916
630.4861046080796
627.4049730300028
627.3955467189013
627.4322194886231
624.3047615385451
625.1484309228865
624.2135689536924
624.2104961859068
621.0487661651331
621.0629038370313
620.7512149718451
620.7676930359595
620.7931347697512
620.8129165225009
620.1255609377846
{'bedrooms': 826.8186291735684, 'longitude': 72.26775688006978, 'doorman': 87.2575575636663, 'fitness_center': 32.140437370836025, 'laundry_in_unit': 1.0838517216317314, 'dining_room': 1.9912820237744882, 'roof_deck': 1.5916390055105012, 'swimming_pool': 3.0811315780767927, 'latitude': 3.100211491457685, 'wheelchair_access': 3.25599537341202}


Not better than our previous baseline. We ended up with an empty dict because no subset of features scored better than the entire set of numberical features. But from this we can conclude the top 4 features offered the most significant boost to the score, so we can use those. Lets see what they are.

In [17]:
impact_table.head(4)

bathrooms     826.818629
bedrooms      865.715355
longitude     989.048700
doorman      1002.990621
dtype: float64

These features make sense. The number of bed/baths significantly affects the value of a residence. There may even be a correalation there with respect to square footage, of which is a key indicator in pricing. Doorman is a luxary accomodation, so will aslo affect pricing. And longitude will determine which borrow the appartment is listed in. From east to west there is Staten Island, Manhattan, Brooklyn, The Bronx, and Queens. These boroughs appear to be more affected by this metric than latitude.  From the output above, we can already determine the score to be $635.15

###Non-numericals

In [18]:
#extract and look at objects in df
features = list(df.dtypes[df.dtypes == 'object'].index)
features

['description', 'display_address', 'street_address', 'interest_level']

In [19]:
df[features].head()

,description,display_address,street_address,interest_level
created,,,,
2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,792 Metropolitan Avenue,medium
2016-06-12 12:19:27,,Columbus Avenue,808 Columbus Avenue,low
2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,241 W 13 Street,high
2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,333 East 49th Street,low
2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,500 West 143rd Street,low


It looks like interest_level is the only feature worth using for a simple boost to our model.

In [20]:
#Encode interest_level to model
interest_train = pd.get_dummies(train.interest_level)
interest_test = pd.get_dummies(test.interest_level)

In [21]:
score(interest_train, interest_test)

1028.1232555108727

Not a huge boost. Lets see what it looks like when we add it to our 4 numerical features

###Combinations

In [22]:
#Merge encoded interest_level data with top 4 numerical features
top_4 = impact_table.index[0:5]

X_train = pd.concat([train[top_4], interest_train], axis = 1)
X_test = pd.concat([test[top_4], interest_test], axis = 1)

In [23]:
X_train.head()

,bathrooms,bedrooms,longitude,doorman,fitness_center,high,low,medium
created,,,,,,,,
2016-04-17 03:26:41,1.0,1,-74.0018,0,0,1,0,0
2016-04-18 02:22:02,1.0,1,-73.9677,0,0,0,1,0
2016-04-28 01:32:41,1.0,4,-73.9493,0,0,0,1,0
2016-04-19 04:24:47,2.0,4,-74.0028,0,0,0,0,1
2016-04-27 03:19:56,1.0,2,-73.9660,0,0,0,1,0


In [24]:
new_best = score(X_train,X_test)

That is a pretty significant boost! With just 5(technically 7) features, we were able to bring the MAE down to 606.17. I assume that if we used all the numerical features AND the encoded interest level, we would see another drop of about 10.

## Updated Score

In [25]:
print(new_best)

606.1728028244722


##All Features

In [26]:
#Merge all features together and test

features = list(df.dtypes[df.dtypes != 'object'].index)
features.remove('price')

X_train = pd.concat([train[features], interest_train], axis = 1)
X_test = pd.concat([test[features], interest_test], axis =1)

score(X_train, X_test)

592.2708705689604

With all the numerical features, and the dummy features from interest_level, we are able to bring the score down to 592. Our best yet!

##Feature Trimming


We can start by looking at our extract disctionary from previous sections to get an idea of where to start

In [27]:
extract = pd.Series(extract).transpose().sort_values(ascending = False)
print(extract)

bedrooms             826.818629
doorman               87.257558
longitude             72.267757
fitness_center        32.140437
wheelchair_access      3.255995
latitude               3.100211
swimming_pool          3.081132
dining_room            1.991282
roof_deck              1.591639
laundry_in_unit        1.083852
dtype: float64


In [28]:
#Switched to little x here because the score is low
features = extract.index.to_list()

x_train = pd.concat([train[features], interest_train], axis = 1)
x_test = pd.concat([test[features], interest_test], axis =1)

score(x_train, x_test)

681.1410401417268

It looks like when we try to extract what appear to be primary features, effect isnt as impactful as one would expect. 

##Feature Engineering

###Description Counts


In [29]:
#Lets look at description to engineer some features
df.description.sample(5)

created
2016-06-03 01:16:59    Amazing Deal for Large Two Bedroom in the LOW ...
2016-05-06 06:00:25    GORGEOUS - 4 BEDROOM -2 BATH - IN BRAND NEW GU...
2016-05-13 02:30:26    Large 1 bedroom in new modern luxury building ...
2016-06-02 16:48:16    HUGE PRIME EAST VILLAGE SUNNY 1 BEDROOM APARTM...
2016-04-09 02:12:04    This is very interesting, bright, and characte...
Name: description, dtype: object

In [30]:
df.description.isna().sum()

1421

In [31]:
df.description.fillna(value = ' ', inplace = True)

In [32]:
#Lets see how many words are in each description
df['description_count'] = df.description.str.count('\w')

df.description_count.describe()

count    48300.000000
mean       469.296190
std        310.682372
min          0.000000
25%        263.000000
50%        437.000000
75%        631.000000
max       3651.000000
Name: description_count, dtype: float64

In [33]:
#What does this do for our model?

train = df[df.index.month < 6]
test = df[df.index.month == 6]

y_train = train.pop('price')
y_test = test.pop('price')

In [34]:
feature = 'description_count'

model.fit(train[[feature]], y_train)
y_pred = model.predict(test[[feature]])

print(mean_absolute_error(y_test,y_pred))

1037.2012769558742


Doesnt look all that great. Lets combine with some of our other features to see how it stands. 

In [35]:
X_train = pd.concat([X_train, train.description_count], axis = 1)
X_test = pd.concat([X_test, test.description_count], axis = 1)

score(X_train, X_test)

592.2754297365761

Looks like that was no help at all! Means that its not HOW much was said, but WHAT!

###NLP

Vectorize tokens of the description data


In [36]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /home/unburied/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/unburied/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/unburied/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

class LemmaTokenizer(object):
  def __init__(self):
    self.wnl = WordNetLemmatizer()
  def __call__(self, description):
    return [self.wnl.lemmatize(t) for t in nltk.word_tokenize(description)]

tfidf = TfidfVectorizer(tokenizer = LemmaTokenizer(), lowercase = True)

words = pd.SparseDataFrame(tfidf.fit_transform(df.description),
                           columns = tfidf.get_feature_names(),
                           default_fill_value = 0)

In [66]:
words.index = df.index

In [67]:
train_condition = words.index.month < 6
test_condition = words.index.month == 6

In [71]:
words = words.to_coo()

In [75]:
train.shape

(31515, 33)

In [80]:
w = words.tocsr()

In [83]:
w_train = w[0:31515]

In [100]:
w_test = w[31515:]

In [85]:
model.fit(w_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [101]:
y_pred = model.predict(w_test)

In [102]:
mean_absolute_error(y_test,y_pred)

3626.1536069691742

In [88]:
test.shape

(16785, 33)

In [89]:
words.shape

(48300, 67581)

In [91]:
w_train.shape, train.shape

((31515, 67581), (31515, 33))

In [96]:
w_test.shape, test.shape

((12784, 67581), (16785, 33))